In [1]:
!wget -O moviedataset.zip http://files.grouplens.org/datasets/movielens/ml-latest-small.zip
print('unziping ...')
!unzip -o -j moviedataset.zip 

--2020-12-03 07:27:17--  http://files.grouplens.org/datasets/movielens/ml-latest-small.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 978202 (955K) [application/zip]
Saving to: ‘moviedataset.zip’

moviedataset.zip    100%[===================>] 955.28K  4.42MB/s    in 0.2s    

2020-12-03 07:27:17 (4.42 MB/s) - ‘moviedataset.zip’ saved [978202/978202]

unziping ...
Archive:  moviedataset.zip
  inflating: links.csv               
  inflating: tags.csv                
  inflating: ratings.csv             
  inflating: README.txt              
  inflating: movies.csv              


In [3]:
!pip install surprise

     |████████████████████████████████| 11.8MB 5.5MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp36-cp36m-linux_x86_64.whl size=1670905 sha256=83bfee1406f27ee21cb340355622be4827f61b7d4e3bb941c1281e2b8960e934
  Stored in directory: /root/.cache/pip/wheels/78/9c/3d/41b419c9d2aff5b6e2b4c0fc8d25c538202834058f9ed110d0
Successfully built scikit-surprise


In [4]:
import itertools
import surprise
from collections import defaultdict
import pandas as pd
import numpy as np

In [5]:
movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')

In [6]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [7]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [8]:
df = pd.merge(movies, ratings, on='movieId', how='inner')

In [9]:
df.head()

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1,4.0,964982703
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5,4.0,847434962
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7,4.5,1106635946
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15,2.5,1510577970
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,17,4.5,1305696483


In [10]:
df['year'] = df['title'].str.extract('(\(\d\d\d\d\))',expand=True)
df['year']= df['year'].str.extract('(\d\d\d\d)',expand=True)
df['title'] = df['title'].str.replace('(\(\d\d\d\d\))','')
df['title'] = df['title'].apply(lambda x: x.strip())

In [11]:
df.head()

,movieId,title,genres,userId,rating,timestamp,year
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1,4.0,964982703,1995
1,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,5,4.0,847434962,1995
2,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,7,4.5,1106635946,1995
3,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,15,2.5,1510577970,1995
4,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,17,4.5,1305696483,1995


In [12]:
reader = surprise.Reader(rating_scale=(0.5, 5))

In [13]:
data = surprise.Dataset.load_from_df(df[['userId', 'movieId', 'rating']], reader)

In [15]:
from surprise.model_selection import train_test_split

In [16]:
trainSet, testSet = train_test_split(data, test_size=.25)

In [17]:
from surprise import SVD

In [18]:
algo = SVD(random_state=0)

In [19]:
algo.fit(trainSet)

In [20]:
prediction = algo.test(testSet)

In [21]:
def MAE(predictions):
        return surprise.accuracy.mae(predictions, verbose=False)
def RMSE(predictions):
        return surprise.accuracy.rmse(predictions, verbose=False)

In [22]:
print('Mean Absolute Error: ',MAE(prediction))
print('Root Mean Squre Error: ',RMSE(prediction))

Mean Absolute Error:  0.6707846882627977
Root Mean Squre Error:  0.8735149887215358


In [23]:
prediction[0]

Prediction(uid=414, iid=3243, r_ui=2.0, est=2.834576612419428, details={'was_impossible': False})

In [24]:
def GetTopN(predictions, n=10, minimumRating=4.0):
    topN = defaultdict(list)
    for userID, movieID, actualRating, estimatedRating, _ in predictions:
        if (estimatedRating >= minimumRating):
            topN[int(userID)].append((int(movieID), estimatedRating))

    for userID, ratings in topN.items():
        ratings.sort(key=lambda x: x[1], reverse=True)
        topN[int(userID)] = ratings[:n]

    return topN

In [25]:
from surprise.model_selection import LeaveOneOut

In [27]:
LOOCV = LeaveOneOut(n_splits=1)

for trainSet, testSet in LOOCV.split(data):
    algo.fit(trainSet)
    
    # Predicts ratings for left-out ratings only
    leftOutPredictions = algo.test(testSet)
    
    # Build predictions for all ratings not in the training set
    bigTestSet = trainSet.build_anti_testset()
    allPredictions = algo.test(bigTestSet)
    # Compute top 10 recs for each user
    topNPredicted = GetTopN(allPredictions, n=10)

In [30]:
def HitRate(topNPredicted, leftOutPredictions):
    hits = 0
    total = 0

 # For each left-out rating
    for leftOut in leftOutPredictions:
        userID = leftOut[0]
        leftOutMovieID = leftOut[1]
        # Is it in the predicted top 10 for this user?
        hit = False
        for movieID, predictedRating in topNPredicted[int(userID)]:
            if (int(leftOutMovieID) == int(movieID)):
                hit = True
                break
        if (hit) :
            hits += 1

        total += 1

    # Compute overall precision
    return hits/total
print("\nHit Rate: ", HitRate(topNPredicted, leftOutPredictions))


Hit Rate:  0.02459016393442623
